In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
import os
from tqdm import tqdm_notebook
from skimage.transform import resize
from keras.preprocessing.image import load_img

Using TensorFlow backend.


In [2]:
path = '/media/danil/Data/Datasets/MTFL'
BATCH_SIZE = 32

In [3]:
training = pd.read_csv(os.path.join(path,'training.txt'), delimiter=" ", header=None)
training.drop(columns=[0], inplace=True)
training.columns = ['path', 'x1', 'x2', 'x3', 'x4', 'x5', 'y1', 'y2', 'y3', 'y4', 'y5', 'gender', 'smile', 'wearing glasses', 'head pose']
training.smile.fillna(2, inplace=True)
training.head()

,path,x1,x2,x3,x4,x5,y1,y2,y3,y4,y5,gender,smile,wearing glasses,head pose
0,lfw_5590\Aaron_Eckhart_0001.jpg,107.25,147.75,126.25,106.25,140.75,108.75,113.25,143.75,158.75,162.75,1.0,2.0,2.0,3.0
1,lfw_5590\Aaron_Guiel_0001.jpg,101.25,146.75,125.25,93.75,139.75,112.25,117.75,137.75,160.75,164.75,1.0,1.0,2.0,3.0
2,lfw_5590\Aaron_Peirsol_0001.jpg,107.75,147.75,130.25,109.25,147.75,114.25,114.25,140.75,154.25,153.75,1.0,1.0,2.0,3.0
3,lfw_5590\Aaron_Pena_0001.jpg,102.75,146.25,126.25,120.75,147.25,117.75,104.75,137.75,164.75,156.75,1.0,2.0,2.0,3.0
4,lfw_5590\Aaron_Sorkin_0001.jpg,102.25,144.75,130.75,100.25,143.25,113.25,112.75,138.75,156.25,155.75,1.0,1.0,2.0,3.0


In [4]:
training.smile = training.smile.map({2:0, 1:1})
training['path'] = training['path'].apply(lambda x: str(x).replace("\\", "/"))

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
X_train_cv = skf.split(training.values, training['smile'].values)

index_train_dict = {}
index_test_dict = {}
for i in range(5):
    ind = next(X_train_cv)
    index_train_dict['split_{}'.format(i)] = ind[0]
    index_test_dict['split_{}'.format(i)] = ind[1]

In [6]:
from sklearn.utils import shuffle

In [7]:
def train_generator(train_id, training, batch_size = BATCH_SIZE):
    training = training[training.index.isin(index_train_dict['split_0'])]
    training = shuffle(training)
    out_rgb = []
    out_label = []
    while True:
        for path_img, smile in (training[['path', 'smile']].values):
            out_rgb += [resize(imread(os.path.join(path, path_img)), (224,224,3))]
            out_label += [smile]
            if len(out_rgb)>=batch_size:
                yield np.stack(out_rgb, 0), np.stack(out_label, 0)
                out_rgb, out_label=[], []

In [8]:
train_gen = train_generator(index_train_dict['split_0'], training)
val_gen = train_generator(index_test_dict['split_0'], training)

# Build Model

In [9]:
from keras.applications import NASNetMobile
model = NASNetMobile(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(2048, activation="elu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Dense(1024, activation="elu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
predictions = Dense(1, activation="sigmoid", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)

from keras.models import Model
# creating the final model 
model_final = Model(input = model.input, output = predictions)

/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [10]:
from keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.models import Model

from keras import backend as K
import tensorflow as tf
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc
  
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model 
model_final.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.0001, decay=0.0001), metrics=['accuracy',auc,f1])

In [11]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

weight_path='NashNet.h5'

checkpoint = ModelCheckpoint(weight_path, monitor='val_auc', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only=False)

reduceLROnPlat = ReduceLROnPlateau(monitor='f1', factor=0.1, patience=3, 
                                   verbose=1, mode='max', min_lr=0.0000001)

def exp_decay(epoch):
    initial_lrate = 0.0001
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    return lrate
lrate = LearningRateScheduler(exp_decay)

callbacks_list = [checkpoint, lrate]

In [12]:
loss_history = [model_final.fit_generator(train_gen, 
                             steps_per_epoch = 8000/BATCH_SIZE, 
                             epochs = 10,
                             validation_data = val_gen,
                             validation_steps = 2000/BATCH_SIZE,
                             callbacks = callbacks_list)]

Epoch 1/10


/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


250/250 [==============================] - 187s 747ms/step - loss: 1.2832 - acc: 0.6429 - auc: 0.6307 - f1: 0.6687 - val_loss: 0.9531 - val_acc: 0.8065 - val_auc: 0.7168 - val_f1: 0.8285

Epoch 00001: val_auc improved from -inf to 0.71682, saving model to NashNet.h5
Epoch 2/10
250/250 [==============================] - 141s 566ms/step - loss: 0.7890 - acc: 0.8830 - auc: 0.8003 - f1: 0.8928 - val_loss: 0.7472 - val_acc: 0.8958 - val_auc: 0.8584 - val_f1: 0.9072

Epoch 00002: val_auc improved from 0.71682 to 0.85839, saving model to NashNet.h5
Epoch 3/10
250/250 [==============================] - 142s 567ms/step - loss: 0.6292 - acc: 0.9471 - auc: 0.8926 - f1: 0.9516 - val_loss: 0.7023 - val_acc: 0.9137 - val_auc: 0.9166 - val_f1: 0.9215

Epoch 00003: val_auc improved from 0.85839 to 0.91659, saving model to NashNet.h5
Epoch 4/10
250/250 [==============================] - 145s 580ms/step - loss: 0.5495 - acc: 0.9735 - auc: 0.9324 - f1: 0.9754 - val_loss: 0.5652 - val_acc: 0.9653 - val_au

KeyboardInterrupt: 

In [13]:
path_test = '/media/danil/Data/Datasets/example_data'
test_images = os.listdir(os.path.join(path_test,'images'))
test_images = [i for i in test_images if 'jpg' in i]

test_smile = os.listdir(os.path.join(path_test,'smile'))
test_smile = [i for i in test_smile if 'jpg' in i]

test_open_mouth = os.listdir(os.path.join(path_test,'open_mouth'))
test_open_mouth = [i for i in test_open_mouth if 'jpg' in i]

In [14]:
test_mouth = []
for i in test_images:
    if i in test_open_mouth:
        test_mouth += [1]
    else:
        test_mouth += [0]
sum(test_mouth)

38

In [15]:
test_smiles = []
for i in test_images:
    if i in test_smile:
        test_smiles += [1]
    else:
        test_smiles += [0]
sum(test_smiles)

23

In [16]:
test_df = pd.DataFrame()
test_df['image'] = test_images
test_df['smile'] = test_smiles
test_df['mouth'] = test_smiles

In [17]:
test_images = [resize(imread(os.path.join(path_test,'images', i)), (224,224,3)) for i in test_images]

/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [18]:
from sklearn.metrics import roc_auc_score, f1_score
predict = model_final.predict(np.stack(test_images, 0), batch_size=1)

In [19]:
score_smile = roc_auc_score(test_df.smile.values, predict)
f1_score_smile = f1_score(test_df.smile.values, np.round(predict))

In [20]:
print(f1_score_smile)

0.1564625850340136
